Inspired in https://github.com/ljchang/dartbrains/tree/master

### Functional Connectivity

In [ ]:
#We're going to calculate temporal correlation among Region of Interest (ROI) 7
#This is tipically done by extracting the temporal response from a seed voxel 
#or the average response from a seed region. Then this time course is regressed 
#against all other voxels in the brain to produce a whole brain map of anywhere 
#that shares a similar time course to the seed.


In [ ]:
#Importing libraries
%matplotlib inline

import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nltools.data import Brain_Data, Design_Matrix, Adjacency
from nltools.mask import expand_mask, roi_to_brain
from nltools.stats import zscore, fdr, one_sample_permutation
from nltools.file_reader import onsets_to_dm
from nltools.plotting import component_viewer
from scipy.stats import binom, ttest_1samp
from sklearn.metrics import pairwise_distances
from copy import deepcopy
import networkx as nx
from nilearn.plotting import plot_stat_map, view_img_on_surf
from bids import BIDSLayout, BIDSValidator
import nibabel as nib


In [ ]:
base_dir = '/home/paosoriom/Universidad/Master Thesis/dev_thesis_SEEG/'
data_dir = os.path.join(base_dir, 'data', 'Localizer')
layout = BIDSLayout(data_dir, derivatives=True)
localizer_path=data_dir
#Data extracted from doi:10.1016/j.neuroimage.2015.09.052",

In [ ]:
ds = dl.Dataset(data_dir)

In [ ]:
results=ds.status(annex='all')

In [ ]:
# file_list = glob.glob(os.path.join(localizer_path, '*', 'fmriprep', '*', 'func', '*tsv'))
file_list = glob.glob(os.path.join(localizer_path, 'derivatives', 'fmriprep', '*', 'func', '*task-localizer_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'))
file_list.sort()
file_list[:10]

In [ ]:
#Prueba
result = ds.get(file_list[0])
result

In [ ]:
#Datos necesarios para el ejercicio 
result = ds.get(os.path.join(localizer_path, 'sub-S01'))
result = ds.get(glob.glob(os.path.join(localizer_path, '*.json')))
result = ds.get(glob.glob(os.path.join(localizer_path, '*.tsv')))
result = ds.get(glob.glob(os.path.join(localizer_path, 'phenotype')))

In [ ]:
sub = 'S01'
fwhm=6

data = Brain_Data(layout.get(subject=sub, task='localizer', scope='derivatives', suffix='bold', extension='nii.gz', return_type='file')[0])
smoothed = data.smooth(fwhm=fwhm)

In [ ]:
smoothed.iplot()

In [ ]:
smoothed

In [ ]:
#we will be using a whole brain parcellation based on similar patterns of
#  coactivation across over 10,000 published studies available in 
# neurosynth (DOI:10.1523/JNEUROSCI.4402-15.2016).
#  We will be using a parcellation of 50 different 
# functionally similar ROIs.
mask = Brain_Data('https://neurovault.org/media/images/8423/k50_2mm.nii.gz')

mask.plot()

In [ ]:
mask_x = expand_mask(mask)

f = mask_x[0:5].plot()

In [ ]:
#Using the vmPFC mask (ROI =32) to seed in a functional connectivity 
# analysis

vmpfc = smoothed.extract_roi(mask = mask_x[32])

plt.figure (figsize = (15,5))
plt.plot(vmpfc, linewidth = 2, color = 'black')
plt.title('vmPFC time series', fontsize = 20)
plt.xlabel('Time (TR)', fontsize = 15)

mask_x[32].plot()

In [ ]:
#Let's now build a regression matrix to perform the WHOLE-BRAIN 
#functional connectivity analysis 

tr = layout.get_tr()
fwhm = 6
n_tr = len(data)

def make_motion_covariates (mc,tr):
    z_mc = zscore(mc)
    all_mc = pd.concat([z_mc, z_mc**2, z_mc.diff(),z_mc.diff()**2],axis=1)
    all_mc.fillna(0,inplace=True)
    return Design_Matrix(all_mc, sampling_freq=1/tr)

vmpfc_z = zscore(pd.DataFrame(vmpfc, columns=['vmpfc']))

csf_mask = Brain_Data(os.path.join(base_dir, 'mask', 'csf.nii.gz'))
csf_mask.plot()
csf_mask = csf_mask.threshold(upper = 0.7, binarize=True)
csf = zscore(pd.DataFrame(smoothed.extract_roi(mask=csf_mask).T, columns=['csf']))


spikes = smoothed.find_spikes(global_spike_cutoff=3, diff_spike_cutoff=3)
covariates = pd.read_csv(layout.get(subject=sub, scope='derivatives', extension='.tsv')[0].path, sep='\t')
mc = covariates[['trans_x','trans_y','trans_z','rot_x', 'rot_y', 'rot_z']]
mc_cov = make_motion_covariates(mc, tr)
dm = Design_Matrix(pd.concat([ 
pd.Series(vmpfc), csf, mc_cov, spikes.drop(labels='TR', axis=1)], axis=1), sampling_freq=1/tr)
dm = dm.add_poly(order=2, include_lower=True)

smoothed.X = dm
stats = smoothed.regress()

vmpfc_conn = stats['beta'][0]

In [ ]:
vmpfc_conn.plot()

In [ ]:
vmpfc_conn.iplot()

## Analysis considering Psychophysiological Interactions

TYPE OF ANALYSIS PROPOSED BY 1053-8119/97 $25.00

In [ ]:
nib.load(layout.get(subject='S01', scope='raw', suffix='bold')[0].path)

In [ ]:
def load_bids_events(layout, subject):
    '''Create a design_matrix instance from BIDS event file'''
    
    tr = layout.get_tr()
    n_tr = nib.load(layout.get(subject=subject, scope='raw', suffix='bold')[0].path).shape[-1]

    onsets = pd.read_csv(layout.get(subject=subject, suffix='events')[0].path, sep='\t')
    onsets.columns = ['Onset', 'Duration', 'Stim']
    return onsets_to_dm(onsets, sampling_freq=1/tr, run_length=n_tr)

dm = load_bids_events(layout, 'S01')
motor_variables = ['video_left_hand','audio_left_hand', 'video_right_hand', 'audio_right_hand']
ppi_dm = dm.drop(motor_variables, axis=1)
ppi_dm['motor'] = pd.Series(dm.loc[:, motor_variables].sum(axis=1))
ppi_dm_conv = ppi_dm.convolve()
ppi_dm_conv['vmpfc'] = vmpfc
ppi_dm_conv['vmpfc_motor'] = ppi_dm_conv['vmpfc']*ppi_dm_conv['motor_c0']
dm = Design_Matrix(pd.concat([ppi_dm_conv, csf, mc_cov, spikes.drop(labels='TR', axis=1)], axis=1), sampling_freq=1/tr)
dm = dm.add_poly(order=2, include_lower=True)

dm.heatmap()

In [ ]:
#Regression analysis and to inspect the interaction term to find regions where 
# the connectivty profile changes as a function of the motor task
smoothed.X = dm
ppi_stats = smoothed.regress()

vmpfc_motor_ppi = ppi_stats['beta'][int(np.where(smoothed.X.columns=='vmpfc_motor')[0][0])]

vmpfc_motor_ppi.plot()


In [ ]:
#Which regions are more functionally connected with the vmPFC during the motor 
# conditions 
vmpfc_motor_ppi.iplot()

## Using PCA 

In [ ]:
#Here we are trying to explain the variance-covariance structure of a high-dimensional 
#random vector. PCA help us to find spatial maps or eigenimages
#We can start with the residuals of our previous regression, which is the 
# remaining signal after removing any variance associated with our covariates
smoothed_denoised=stats['residual']


In [ ]:
n_components = 10

pca_stats_output = smoothed_denoised.decompose(algorithm='pca', axis='images', n_components=n_components)


In [ ]:
component_viewer(pca_stats_output, tr=layout.get_tr())


In [ ]:
f,a = plt.subplots(ncols=2, figsize=(12, 5))
a[0].plot(pca_stats_output['decomposition_object'].explained_variance_ratio_)
a[0].set_ylabel('Percent Variance Explained', fontsize=18)
a[0].set_xlabel('Component', fontsize=18)
a[0].set_title('Variance Explained', fontsize=18)
a[1].plot(np.cumsum(pca_stats_output['decomposition_object'].explained_variance_ratio_))
a[1].set_ylabel('Percent Variance Explained', fontsize=18)
a[1].set_xlabel('Component', fontsize=18)
a[1].set_title('Cumulative Variance Explained', fontsize=18)

## Approach using graph theory 

In [ ]:
#Extracting the average time course within each ROI from the 50 parcels 
#From the denoised data
rois = smoothed_denoised.extract_roi(mask=mask)

plt.figure(figsize=(15,5))
plt.plot(rois.T)
plt.ylabel('Mean Intensitiy', fontsize=18)
plt.xlabel('Time (TRs)', fontsize=18)

In [ ]:
#Calculate the edges of the nodes using pearson correlations. 
roi_corr = 1 - pairwise_distances(rois, metric='correlation')

sns.heatmap(roi_corr, square=True, vmin=-1, vmax=1, cmap='RdBu_r')

In [ ]:
#To create a binary matrix, we use an arbitrary threshold in the correlation 
#matrix
a = Adjacency(roi_corr, matrix_type='similarity', labels=[x for x in range(50)])
a_thresholded = a.threshold(upper=.6, binarize=True)

a_thresholded.plot()

In [ ]:
plt.figure(figsize=(20,15))
G = a_thresholded.to_graph()
pos = nx.kamada_kawai_layout(G)
node_and_degree = G.degree()
nx.draw_networkx_edges(G, pos, width=3, alpha=.2)
nx.draw_networkx_labels(G, pos, font_size=14, font_color='darkslategray')

nx.draw_networkx_nodes(G, pos, nodelist=list(dict(node_and_degree).keys()),
                       node_size=[x[1]*100 for x in node_and_degree],
                       node_color=list(dict(node_and_degree).values()),
                       cmap=plt.cm.Reds_r, linewidths=2, edgecolors='darkslategray', alpha=1)


In [ ]:
plt.hist(dict(G.degree).values())
plt.ylabel('Frequency', fontsize=18)
plt.xlabel('Degree', fontsize=18)

In [ ]:
degree = pd.Series(dict(G.degree()))
brain_degree = roi_to_brain(degree, mask_x)
brain_degree.plot()

In [ ]:
view_img_on_surf(brain_degree.to_nifti())


In [ ]:
mask_x[16].plot()